# **How to ingest data to Pinecone using LangChain**

- This notebook shows you how you can ingest data from a document into a Pinecone Database.
- It uses Langchain to ease the process of split and ingest the data

### **1. Import dependencies**

In [1]:
import os
import pinecone
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chat_models import AzureChatOpenAI
from langchain.chains.question_answering import load_qa_chain

e:\Coding\ai\building-qa-app-with-openai-pinecone-and-streamlit\.venv\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### **2. Read PineCone and Azure OpenAI environment variables**

In [8]:
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_API_ENV = os.getenv('PINECONE_ENVIRONMENT')
PINECONE_INDEX_NAME =  os.getenv('PINECONE_INDEX_NAME')
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_APIKEY')
AZURE_OPENAI_API_BASE = os.getenv('AZURE_OPENAI_BASE_URI')
AZURE_OPENAI_API_TYPE = 'azure'
AZURE_OPENAI_API_VERSION = '2023-03-15-preview'
AZURE_OPENAI_EMBEDDINGS_MODEL_NAME= os.getenv('AZURE_OPENAI_EMBEDDINGS_MODEL_NAME')
AZURE_OPENAI_GPT4_MODEL_NAME= os.getenv('AZURE_OPENAI_GPT4_MODEL_NAME')

### **3. Load your data files and split it into chunks**

In [3]:
loader = UnstructuredPDFLoader("./docs/NET-Microservices-Architecture-for-Containerized-NET-Applications.pdf")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunks = text_splitter.split_documents(data)

print (f'You have a total of {len(chunks)} chunks')

detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.


You have a total of 917 chunks


### **4. Init Azure OpenAI and Pinecone clients**

In [4]:
embeddings = OpenAIEmbeddings(
    openai_api_base=AZURE_OPENAI_API_BASE, 
    openai_api_key=AZURE_OPENAI_API_KEY, 
    openai_api_type=AZURE_OPENAI_API_TYPE,
    model=AZURE_OPENAI_EMBEDDINGS_MODEL_NAME, 
    chunk_size=1)

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_API_ENV  
)

### **5. Create and store embeddings into Pinecone**

In [6]:
docsearch = Pinecone.from_texts([t.page_content for t in chunks], embeddings, index_name=PINECONE_INDEX_NAME)

### **6. Ask a question**

In [10]:
llm = AzureChatOpenAI(
    temperature=0, 
    openai_api_base=AZURE_OPENAI_API_BASE, 
    openai_api_key=AZURE_OPENAI_API_KEY, 
    openai_api_version=AZURE_OPENAI_API_VERSION, 
    deployment_name=AZURE_OPENAI_GPT4_MODEL_NAME)

chain = load_qa_chain(llm, chain_type="stuff")

query = "Enumerate 3 strategies to handle partial failures?"
docs = docsearch.similarity_search(query, include_metadata=True)

result = chain.run(input_documents=docs, question=query)

print(f"Answer: \n\n {result}")

Answer: 

 1. Use asynchronous communication: Implement message-based communication across internal microservices to minimize ripple effects and enforce a higher level of microservice autonomy.

2. Implement retries with exponential backoff: In case of transient faults, use the "Retry pattern" to retry the operation with increasing time intervals between attempts, allowing the system to recover.

3. Use the Circuit Breaker pattern: Track the number of failed requests, and if the error rate exceeds a configured limit, trip the circuit breaker to prevent further attempts. After a timeout period, try again and close the circuit breaker if new requests are successful.
